In [0]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

src = cv2.imread('H:/499B/ALL DATA/Original/00a02700-2ea2-4590-9e15-ffc9160fd3de.png')
img = src.copy()
gray = img[:, :, 0]
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)


# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 3) # Morphological open operation

# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=9)

# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3)
ret, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)


# Marker labelling
ret, markers1 = cv2.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers1+10

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers3 = cv2.watershed(img,markers)
img[markers3 == -1] = [255,255,255]

plt.subplot(241), plt.imshow(cv2.cvtColor(src, cv2.COLOR_BGR2RGB)),
plt.title('Original'), plt.axis('off')
plt.subplot(242), plt.imshow(thresh, cmap='gray'),
plt.title('Threshold'), plt.axis('off')
plt.subplot(243), plt.imshow(sure_bg, cmap='gray'),
plt.title('Dilate'), plt.axis('off')
plt.subplot(244), plt.imshow(dist_transform, cmap='gray'),
plt.title('Dist Transform'), plt.axis('off')
plt.subplot(245), plt.imshow(sure_fg, cmap='gray'),
plt.title('Threshold'), plt.axis('off')
plt.subplot(246), plt.imshow(unknown, cmap='gray'),
plt.title('Unknow'), plt.axis('off')
plt.subplot(247), plt.imshow(np.abs(markers), cmap='jet'),
plt.title('Markers'), plt.axis('off')
plt.subplot(248), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)),
plt.title('Result'), plt.axis('off')

plt.show()
gr = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret1,th1 = cv2.threshold(gr,127,255,0)
#ret1,th1 = cv2.threshold(gr,185,200,cv2.THRESH_BINARY_INV)
#cv2.imshow('unknown',th1)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


imagem = th1
imagem = cv2.bitwise_not(imagem)
cv2.imshow('unknown',imagem)

cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('C:/Users/DIPTO/Desktop/try.png', imagem)